In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

!pip install pytextrank
!pip install spacy
!python -m spacy download en_core_web_lg
import spacy
import pytextrank
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.7 MB/s eta 0:00:000:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
/usr/local/lib/python3.10/dist-packages
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


****Dataset loading****

In [6]:
Validation_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv")
Train_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
Test_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")

****A bit of data exploration****

In [3]:
#Train_data.head()
#Test_data.head()
print(Train_data.iloc[0])
print(Test_data.iloc[0])
print("Describtion",Train_data.describe)

id                     0001d1afc246a7964130f43ae940af6bc6c57f01
article       By . Associated Press . PUBLISHED: . 14:11 EST...
highlights    Bishop John Folda, of North Dakota, is taking ...
Name: 0, dtype: object
id                     92c514c913c0bdfe25341af9fd72b29db544099b
article       Ever noticed how plane seats appear to be gett...
highlights    Experts question if  packed out planes are put...
Name: 0, dtype: object
Describtion <bound method NDFrame.describe of                                               id  \
0       0001d1afc246a7964130f43ae940af6bc6c57f01   
1       0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2       00027e965c8264c35cc1bc55556db388da82b07f   
3       0002c17436637c4fe1837c935c04de47adb18e9a   
4       0003ad6ef0c37534f80b55b4235108024b407f0b   
...                                          ...   
287108  fffdfb56fdf1a12d364562cc2b9b1d4de7481dee   
287109  fffeecb8690b85de8c3faed80adbc7a978f9ae2a   
287110  ffff5231e4c71544bc6c97015cdb16c60e42b3f4   
2871

# Text Cleaning Function

In this section, we define a simple text cleaning function called `clean_text()`.  
The purpose of this function is to:

- Handle missing values (`NaN`)
- Remove extra whitespace between words
- Remove leading and trailing spaces

This helps us ensure that the text data is clean and consistent before applying any further processing.


In [7]:
def clean_text(text):
    """
    Clean the input text by:
    - Handling missing (NaN) values
    - Removing extra spaces
    - Trimming leading and trailing spaces
    
    Args:
        text (str): The input text to clean.

    Returns:
        str: The cleaned text.
    """
    if pd.isna(text):  # Handle missing values
        return ""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.strip()                # Trim leading/trailing spaces
    return text
test_data = Test_data.copy()
test_data['article'].apply(clean_text)


In [11]:
test_data.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


### 🛠️ Using SpaCy's TextRank for Extractive Summarization

We load the SpaCy model `en_core_web_lg` and add the TextRank algorithm to it. TextRank helps identify key sentences in the text for extractive summarization.


In [8]:


#define the model
nlp = spacy.load("en_core_web_lg")

# add the textrank algorithm to the model
nlp.add_pipe("textrank")


# Text Summarization with TextRank

We summarize each article in the dataset using the TextRank algorithm:

- Process each article with an NLP model.
- Extract the top 2 most important sentences.
- Save the summaries in a new column called `"processed_text"`.

This helps reduce the size of the articles while keeping their key information.


In [7]:
# Create an empty list to store processed text
processed_articles = []

# Iterate over each row in the "article" column
for text in test_data["article"]:
    doc = nlp(text)  # Apply NLP processing
    summary = " ".join([sent.text for sent in doc._.textrank.summary(limit_sentences=2)])
    processed_articles.append(summary)

# Add the processed text as a new column in the dataframe
test_data["processed_text"] = processed_articles

# Print an example processed article
#print(test_data["processed_text"].iloc[0])


In [8]:
print(test_data["processed_text"].iloc[0])
print("\n")
print("\n")
print(Test_data["article"].iloc[0])


While United Airlines has 30 inches of space, Gulf Air economy seats have between 29 and 32 inches, Air Asia offers 29 inches and Spirit Airlines offers just 28 inches. But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased, reported the Detroit News.




Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a mini

# Abstractive Summarization using PEGASUS

We use the **PEGASUS** model to generate abstractive summaries, meaning it creates new sentences instead of copying from the text.  
PEGASUS is a transformer-based model trained to produce human-like summaries by understanding the main idea of the input.


In [9]:
# install and import important libraries
!pip install transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Loading the PEGASUS Model

We load the pre-trained **PEGASUS** model and its tokenizer from Hugging Face.

- `google/pegasus-cnn_dailymail` is a version fine-tuned for summarizing news articles.
- The tokenizer prepares the text input for the model.
- The model generates abstractive summaries based on the processed input.


In [10]:
# pick the pre_trained model (from huggingface model hub)
model_name = "google/pegasus-cnn_dailymail"

# load the tokenizers
tokenizer = PegasusTokenizer.from_pretrained(model_name)

# define pegasus model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# Summarizing Articles in Batches

To efficiently summarize articles without running into memory issues:

- We copy a small subset of the data (first 100 articles) and clean the text.
- We move the model to GPU (if available) for faster processing.
- We define a batching function that:
  - Tokenizes multiple articles at once
  - Generates summaries using the PEGASUS model
  - Decodes the model outputs back into readable text

Finally, we store the generated summaries in a new column called `"summary"`.


In [17]:
import torch
from tqdm import tqdm  # for showing progress

# 1. Use only the first 10 rows to avoid memory problems
test_data2 = Test_data.copy().head(100)
test_data2['article'] = test_data2['article'].apply(clean_text)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pegasus_model = pegasus_model.to(device)

# Batching function
def generate_summaries_in_batches(texts, batch_size=4):
    summaries = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        # Tokenize
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            max_length=512,
            return_tensors="pt"
        ).to(device)

        # Generate summaries
        with torch.no_grad():
            summary_ids = pegasus_model.generate(
                **inputs,
                max_length=200,  # try a smaller summary length for less memory
                num_beams=4,
                early_stopping=True
            )

        # Decode
        decoded = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]
        summaries.extend(decoded)
    
    return summaries

# Generate summaries
summaries = generate_summaries_in_batches(test_data2['article'].tolist(), batch_size=4)

# Store results
test_data2["summary"] = summaries

100%|██████████| 25/25 [01:06<00:00,  2.64s/it]


In [18]:
print(test_data2["summary"].iloc[5])
print("\n")
print("\n")
print(test_data2["article"].iloc[5])


Pigwig the pig fell into a swimming pool in an upmarket neighbourhood .<n>The prize porker was unable to get out of the water and had to be rescued .<n>Firefighters also had to winch a horse out of a swimming pool in West Sussex .




This is the moment that a crew of firefighters struggled to haul a giant pig out of a garden swimming pool. The prize porker, known as Pigwig, had fallen into the pool in an upmarket neighbourhood in Ringwood, Hampshire. His owners had been taking him for a walk around the garden when the animal plunged into the water and was unable to get out. A team from Dorset Fire and Rescue struggled to haul the huge black pig out of swimming pool water . The prize porker known as Pigwig had fallen into the water and had then been unable to get out again . Two fire crews and a specialist animal rescue team had to use slide boards and strops to haul the huge black pig from the small pool. A spokesman for Dorset Fire and Rescue Service said: 'At 4.50pm yesterday the se